### 네이버 '갤러리아 백화점' 키워드 문서 수집
#### 블로그 수집
#### 최신 순 수집 

In [4]:
from urllib import request
resp  = request.urlopen('https://www.google.com')
resp.geturl()

from urllib import request
# URL 분석
# anlasisURL = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=post&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%EC%95%84+%EB%B0%B1%ED%99%94%EC%A0%90&oquery=%EA%B0%A4%EB%9F%AC%EB%A6%AC%EC%95%84+%EB%B0%B1%ED%99%94%EC%A0%90&tqi=UfRpRdprvmZsslNn0usssssstEl-039018'
# anlasisURL = 'https://search.naver.com/search.naver?where=post&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%EC%95%84%20%EB%B0%B1%ED%99%94%EC%A0%90&st=sim&sm=tab_opt&date_from=20191001&date_to=20191031&date_option=8&srchby=all&dup_remove=1&post_blogurl=&post_blogurl_without=&nso=so%3Ar%2Ca%3Aall%2Cp%3Afrom20191001to20191031'
anlasisURL = 'https://search.naver.com/search.naver?date_from=20191001&date_option=8&date_to=20191031&dup_remove=1&nso=p%3Afrom20191001to20191031&post_blogurl=&post_blogurl_without=&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%EC%95%84%20%EB%B0%B1%ED%99%94%EC%A0%90&sm=tab_pge&srchby=all&st=sim&where=post&start=31'

In [5]:
# URL 분석
from urllib import parse
result = parse.urlparse(anlasisURL)
for i,_ in enumerate(result):
    print("{0}:{1}".format(i,_))

print(result[4])
result[4].split('&')


0:https
1:search.naver.com
2:/search.naver
3:
4:date_from=20191001&date_option=8&date_to=20191031&dup_remove=1&nso=p%3Afrom20191001to20191031&post_blogurl=&post_blogurl_without=&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%EC%95%84%20%EB%B0%B1%ED%99%94%EC%A0%90&sm=tab_pge&srchby=all&st=sim&where=post&start=31
5:
date_from=20191001&date_option=8&date_to=20191031&dup_remove=1&nso=p%3Afrom20191001to20191031&post_blogurl=&post_blogurl_without=&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%EC%95%84%20%EB%B0%B1%ED%99%94%EC%A0%90&sm=tab_pge&srchby=all&st=sim&where=post&start=31


['date_from=20191001',
 'date_option=8',
 'date_to=20191031',
 'dup_remove=1',
 'nso=p%3Afrom20191001to20191031',
 'post_blogurl=',
 'post_blogurl_without=',
 'query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%EC%95%84%20%EB%B0%B1%ED%99%94%EC%A0%90',
 'sm=tab_pge',
 'srchby=all',
 'st=sim',
 'where=post',
 'start=31']

In [7]:
from urllib import parse
parse.unquote('%EA%B0%A4%EB%9F%AC%EB%A6%AC%EC%95%84%20%EB%B0%B1%ED%99%94%EC%A0%90')

'갤러리아 백화점'

### HTML 추출 FUN
#### 입력 : URL, Parameter, retry cnt)

In [4]:
# INPUT URL HTML DOWN FUN
import requests
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
def getDownload(url,params={}, retries=3):
    resp = None    
    try:
        resp = requests.get(url,params, headers= header)
        resp.raise_for_status()
#         print(resp.url)
    except requests.exceptions.HTTPError as e:
        if 500<= e.reponse.status_code < 600 and retries >0:
            print(retries)
            resp = getDownload(url,params,retries-1)
        else:
            print(url)
            print(e.reponse.status_code)
            print(e.reponse.reason)
            print(e.reponse.headers)
    return resp


In [6]:
html = getDownload('https://blog.naver.com/PostView.nhn?blogId=zxq-521&logNo=221637895597&redirect=Dlog&widgetTypeCall=true&directAccess=false')
html.text
dom = BeautifulSoup(html.text,'html.parser')
a = [_.text for _ in dom.select('.se_textarea')]
a

['\n강다니엘, 팬한테 보내는 하트\n',
 '[일요신문] 가수 강다니엘이 3일 오후 서울 압구정동 갤러리아백화점에서 열린 포토월 행사에 참석해 포토타임을 갖고 있다.\xa0\xa0/박정훈 기자 onepark@ilyo.co.kr',
 'http://ilyo.co.kr/?ac=article_view&entry_id=346523',
 '#강다니엘 #KangDaniel #독무 #고양이 #인스타그램 #먹방 #공항패션 #천사들의편지 #이불밖은위험해 #뱅뱅뱅']

### URL 사이트 글 수집 후 파일로 저장
이슈1 : 재귀 함수
1. http://koojungyeon.com/221707351127 
2. http://blog.naver.com/PostList.nhn?blogId=daylily01
3. http://blog.naver.com/PostView.nhn?blogId=daylily01&logNo=221707351127&redirect=Dlog&widgetTypeCall=true&directAccess=false

In [5]:
# 입력 url 사이트 스크랩 파일 저장
from bs4 import BeautifulSoup
from urllib import parse
from collections import defaultdict
from os import path

import os


def craping_to_file(n_url, file_name,p_f_path):
    last_dom = None
    pas = dict()
    
#     file_path = './blog'
    # 파일 저장 경로 설정
    #file_path = './blog/'
    file_path = p_f_path if p_f_path != None else './blog'
#     print(n_url)
    if file_name is None :
        file_name = n_url
    qry = parse.urlparse(n_url).query.strip()
#     print("url:"+n_url.split('?')[0])
    if  len(parse.urlparse(n_url).query) != 0:
        pas = dict(parse.parse_qsl(parse.urlparse(n_url).query))
    html = getDownload(n_url.split('?')[0],pas)
        
    if html.status_code != 200 : return -1
    n_dom = BeautifulSoup(html.text,"html.parser")
    # 블로그는 iframe을 사용함 
    # 단 irame은 두가 case로 분류된(ifram,frame)
#     frm_dom = n_dom.select("iframe") if len(n_dom.select("iframe")) != 0 else n_dom.select("frame")
    frm_dom = n_dom.select("#screenFrame") if len(n_dom.select("#screenFrame")) != 0 else n_dom.select("#mainFrame")
    if len(frm_dom) == 0 :
        last_dom = n_dom
        text_list = [_.text for _ in last_dom.select("span")]
        text_list.extend( [_.text for _ in last_dom.select(".se_textarea")])
#         print(text_list)
        
        #파일 저장
        if path.exists(file_path) == False: os.mkdir('./blog')
        
        #파일 이름 저장
        last_file =""
        if file_name is None or file_name == "":
            if n_url.startswith('http://blog.naver.com') or n_url.startswith('https://blog.naver.com'):
                par =dict(parse.parse_qsl( parse.urlparse(n_url).query))
#                 print( par['blogId'].strip())
#                 print(par['logNo'].strip())
                last_file = par['blogId'].strip()+"_"+par['logNo'].strip()
            else:
                last_file = n_url.replace('https://','').replace('http://','').split('/')[0]            
        
        with open(file_path+'/'+last_file,'w',encoding='utf-8') as fp:
            fp.write('\n'.join(text_list))
            fp.close()
#         print('저장완료:'+last_file)
    elif frm_dom[0].attrs['src'].startswith('/'):
        new_url = "https://blog.naver.com"+frm_dom[0].attrs['src'].strip()
        craping_to_file(new_url,file_name,p_f_path)
#         new_html = getDownload(new_url)
#         last_dom = BeautifulSoup(new_html.text,"lxml")
#         print(last_dom)
    else:
        new_url = frm_dom[0].attrs['src']
#         new_html = getDownload(new_url)
#         print(new_html.text)
        craping_to_file(new_url,file_name,p_f_path)
#         last_dom = BeautifulSoup(new_html.text,"lxml")
#         print(last_dom) 



###  Main Program

In [ ]:
#링크 URL 수집
from bs4 import BeautifulSoup
import timeit

url =  'https://search.naver.com/search.naver'
param = {"sm":"tab_hty.top",
         "where":"post",
         "query":"갤러리아 백화점",
         "date_option":"0",
         "dup_remove":"1",
         "srchby":"all",
         "st":"date",
         "start":"11"
         }


ind_cnt = 100000

url_links = list()
# 링크 URL 수집

print('---------------리스트 생성------------')
time_url_start = timeit.default_timer()
for i in range(ind_cnt):
    key_s = i*10+1
    param['start'] = key_s
    
    resp = getDownload(url,param,3)
    if resp == -1 : continue 
#     print(resp.status_code)
#     if resp.status_code != 200 : break
    dom = BeautifulSoup(resp.text,'lxml')
    links = [_['href'] for _ in dom.select("a.sh_blog_title")]
    url_links.extend(links) 
    if i%10 ==0 :
        print('{}초:{}건'.format( timeit.default_timer()-time_url_start,str(i)))
print('URL 리스트 완료')
with open('./blog/url_list.txt','w') as fp:
    fp.write('\n'.join(url_links))
    fp.close()
# URL 접속 후 TEXT 수집    
j= 1
time_save_start = timeit.default_timer()
print('----------------수집저장-----------')
for _ in url_links:
    try:
        craping_to_file(_,"")
    except Exception as e:
        print('Exception:'+_)
        continue
    if j%10 ==0 : print('{}초:{}건:'.format(timeit.default_timer()-time_save_start,str(j)))
    j+=1
print('파일 저장 완료')
# url 리스트 파일 저장


---------------리스트 생성------------
0.38530429099773755초:0건
4.40020666700002초:10건
9.678700049000327초:20건
15.995084021997172초:30건
23.326722445002815초:40건
31.746687272003328초:50건
40.22226925999712초:60건
48.733996216004016초:70건
57.31323063799937초:80건
66.10053311799857초:90건
74.91840705699724초:100건
83.56999078600347초:110건
92.20441716300411초:120건
100.61805218199879초:130건
109.48435758300184초:140건
117.84722753400274초:150건
126.3067183910025초:160건
134.60533702000248초:170건
142.96108253199782초:180건
151.40638904200023초:190건
160.72021460100223초:200건
169.13069854200148초:210건
177.42697073100135초:220건
185.8230789320005초:230건
194.1193391170018초:240건
202.43924229500408초:250건
210.8593154199989초:260건
219.285748864997초:270건
227.6122577770002초:280건
235.95120573099848초:290건
244.36715634899883초:300건
252.7394752859982초:310건
261.0399302259975초:320건
269.4873261899993초:330건
277.8704821209976초:340건
286.2167626590017초:350건
294.6364047570023초:360건
302.9411104890023초:370건
311.22362233400054초:380건
319.65148488600244초:390건

In [13]:
url_list

['https://blog.naver.com/jbw03145?Redirect=Log&logNo=221707734188\n',
 'https://blog.naver.com/vomvomglass?Redirect=Log&logNo=221706358135\n',
 'https://blog.naver.com/jhlim1298z?Redirect=Log&logNo=221707726696\n',
 'https://blog.naver.com/christpower77?Redirect=Log&logNo=221707723906\n',
 'https://blog.naver.com/dangnaky84454?Redirect=Log&logNo=221707705452\n',
 'https://blog.naver.com/diffi00?Redirect=Log&logNo=221707698124\n',
 'https://blog.naver.com/skyflow89?Redirect=Log&logNo=221703791703\n',
 'https://bug3.blog.me/221707695162\n',
 'https://blog.naver.com/8x0ucder7j1?Redirect=Log&logNo=221707689730\n',
 'https://blog.naver.com/wtl485?Redirect=Log&logNo=221707689386\n',
 'https://blog.naver.com/octo9?Redirect=Log&logNo=221707683867\n',
 'https://blog.naver.com/bd0502?Redirect=Log&logNo=221707682814\n',
 'https://blog.naver.com/n4h-l6kafbt?Redirect=Log&logNo=221707675348\n',
 'https://blog.naver.com/stockone1?Redirect=Log&logNo=221707673450\n',
 'https://blog.naver.com/heimine?Re

-----------------------------------------
이슈2 : 1000건??

https://search.naver.com/search.naver?date_from=20191001&date_option=8&date_to=20191031&dup_remove=1&nso=p%3Afrom20191001to20191031&post_blogurl=&post_blogurl_without=&query=%EA%B0%A4%EB%9F%AC%EB%A6%AC%EC%95%84%20%EB%B0%B1%ED%99%94%EC%A0%90&sm=tab_pge&srchby=all&st=sim&where=post&start=991


#### <수집 프로그램 재개발>
###### 추가 기능 : 입력 월별 데이터 수집
###### 저장 경로 변경 -> 월별로 폴더 생성 

In [ ]:
#링크 URL 수집
from bs4 import BeautifulSoup
import timeit
import pandas as pd
import os
import calendar


# input 
p_start_yymm = '201901'
p_end_yymm = '201912'


start_ymd = p_start_yymm+'01'
end_ymd = p_end_yymm +  str(calendar.monthrange(int(p_end_yymm[:4]),int(p_end_yymm[4:6]))[1])


ind_cnt = 100


for _ in pd.date_range(start=start_ymd,end=end_ymd,dtype='datetime64[ns]',freq='M'):
    
#     p_start_ymd = _.strftime('%Y%m')+'01'
#     p_end_ymd = _.strftime('%Y%m%d')
  
    # 15일씩 분할 수집
    loop_end_ymd = [_.strftime('%Y%m')+'15',_.strftime('%Y%m%d')]
    
    
    for lp_ymd in loop_end_ymd:
        
        p_start_ymd =   lp_ymd[:6]+'01'  if lp_ymd[6:] == '15' else lp_ymd[:6]+'16'
        p_end_ymd = lp_ymd
        print('수집시작:{0}~{1}'.format(p_start_ymd,p_end_ymd))
           
        url =  'https://search.naver.com/search.naver'
    #     param = {"sm":"tab_hty.top",
    #              "where":"post",
    #              "query":"갤러리아 백화점",
    #              "date_from":p_start_ymd,
    #              "date_to":p_end_ymd,
    #              "date_option":"0", # 연관성 우선:0, 날짜순
    #              "dup_remove":"1",
    #              "srchby":"all",
    #              "st":"date",
    #              "start":"11"
    #              }
        param = { "date_from":p_start_ymd,
                 "date_option":"8", # 연관성 우선:0, 날짜순
                 "date_to":p_end_ymd,
                 "dup_remove":"1",
                 "post_blogurl":"",
                 "post_blogurl_without":"",
                 "query":"갤러리아 백화점",
                 "sm":"tab_hty.top",
                 "srchby":"all",
                 "st":"sim",
                 "where":"post",
                 "start":"11"
                 }


        p_file_path = './blog/G_'+lp_ymd[:6]
        p_urlf_FName = 'url_list'+lp_ymd+'.txt'

        url_links = list()
       # 링크 URL 수집

        print('---------------리스트 생성------------')
        time_url_start = timeit.default_timer()
        for i in range(ind_cnt):
            key_s = i*10+1
            param['start'] = key_s

            resp = getDownload(url,param,3)
            if resp == -1 : continue 
        #     print(resp.status_code)
        #     if resp.status_code != 200 : break
            dom = BeautifulSoup(resp.text,'lxml')
            links = [_['href'] for _ in dom.select("a.sh_blog_title")]
            url_links.extend(links) 
            if i%10 ==0 :
                print('{}초:{}건'.format( timeit.default_timer()-time_url_start,str(i)))

        # 디렉터리 확인 
        if os.path.exists(p_file_path) == False:
            os.makedirs(p_file_path)

        with open(p_file_path+'/'+p_urlf_FName,'w') as fp:
            fp.write('\n'.join(url_links))
            fp.close()

        print('URL 리스트 완료')
        # URL 접속 후 TEXT 수집    
        j= 1
        time_save_start = timeit.default_timer()
        print('----------------수집저장-----------')
        for _ in url_links:
            try:
                craping_to_file(_,"",p_file_path)
            except Exception as e:
                print('Exception({0}):{1}'.format(e,_))
                continue
            if j%10 ==0 :print('{}초:{}건:'.format(timeit.default_timer()-time_save_start,str(j)))
            j+=1
        print('파일 저장 완료:{0}'.format(p_start_ymd))
        # url 리스트 파일 저장


수집시작:20190301~20190315
---------------리스트 생성------------
0.42154996099998243초:0건
4.269549770000594초:10건
8.736868481999409초:20건
13.42367160700087초:30건
18.53296887200122초:40건
24.398262660997716초:50건
30.270196485998895초:60건
36.62703313200109초:70건
43.493240440999216초:80건
50.3164145289993초:90건
URL 리스트 완료
----------------수집저장-----------
4.907381968001573초:10건:
9.853760220001277초:20건:
14.452018297000905초:30건:
19.023745967999275초:40건:
23.104316597000434초:50건:
27.643120631000784초:60건:
32.58714240900008초:70건:
38.1898842849987초:80건:
43.655444186999375초:90건:
49.102652280002076초:100건:
53.618855141001404초:110건:
58.461779191999085초:120건:
63.54883272100051초:130건:
68.68306361299983초:140건:
73.70341875799932초:150건:
79.44817565799895초:160건:
85.5350702350006초:170건:
91.64406794800016초:180건:
97.04160612000123초:190건:
103.51053955300085초:200건:
108.20049664899852초:210건:
114.18999038599941초:220건:
120.1555365029999초:230건:
126.21926201799943초:240건:
135.0418490129996초:250건:
140.2333025859989초:260건:
146.136221664000